In [206]:
import math
import csv
import json
import numpy
import random
from collections import Counter
from collections import OrderedDict

In [99]:
class data_transform:
    """
    load the data
    """
    def _init_(self,file_name):
        self.file_name = file_name
    def tran_json(self,file_name):
        f = open(file_name)
        self.data = [json.loads(line.rstrip()) for line in f]
        return self.data

In [137]:
"""
Provide a sample dataset of the origin data, since the origin data is too large and it's alomost 
1GB.
However, the performance will not good enough since the sample data is smaller 
"""
review = data_transform()
raw_data = review.tran_json("yelp_dataset/yelp_academic_dataset_review_sample.json")

user = []
business = []
for rev in raw_data:
    if rev["user_id"] in user:
        pass
    else:
        user.append(rev["user_id"])
    if rev["business_id"] in business:
        pass
    else:
        business.append(rev["business_id"])
business.sort()
user.sort()

In [138]:
print "Total business is %d" %len(business)
print "Total user is %d" %len(user)

Total business is 5036
Total user is 46017


In [108]:
def matrix_user_based(raw_data,business):
    """
    transfrom the data to the matrix 
        the row is business
        the column is users 
        the value is the star
    """
    user_dict = {}
    for rev in raw_data:
        if rev["user_id"] in user_dict:
            user_dict[rev["user_id"]][rev["business_id"]] = int(rev["stars"])
        else:
            user_dict[rev["user_id"]] = {rev["business_id"]:int(rev["stars"])}
    user_dict = OrderedDict(sorted(user_dict.items(),key = lambda x: x[0]))
    print type(user_dict)
    matrix = []
    for user,value in user_dict.items():
        matrix_value = []
        for bus in business:
            if bus in value:
                matrix_value.append(value[bus])
            else:
                matrix_value.append(.0)
        matrix.append(matrix_value)
    return matrix

In [113]:
a = numpy.array(matrix_user_based(raw_data,business))

<class 'collections.OrderedDict'>


In [125]:
a.shape

(46017, 5036)

In [119]:
def similarity_cos(vector_x,vector_y):
    """
    Compute the similarity between the pairs of users using the cosine
    """
    top = sum([x*y for x,y in zip(vector_x,vector_y)])
    bottom = math.sqrt(sum([x**2 for x in vector_x])) * math.sqrt(sum([y**2 for y in vector_y]))
    similarity = float(top)/bottom
    return similarity

In [128]:
matrix = matrix_user_based(raw_data,business)

<class 'collections.OrderedDict'>


In [130]:
len(matrix)

46017

In [211]:
def user_based_cf(custom,user,matrix,n):
    recommend_list = []
    index = user.index(custom) 
    list_sim = []
    for i in xrange(len(user)):
        if i != index:
            sim = similarity_cos(matrix[i],matrix[index])
            list_sim.append([i,sim])
        else:
            continue
    top = [x for x in list_sim if x[1] != 0]
    top = sorted(top,key = lambda x:x[1],reverse = True)

    for key,value in enumerate(matrix[index]):
        if value == 0.0:
            predict_val = 0
            div_sum = 0
            for user_top in top:
                if matrix[user_top[0]][key] != 0:
                    predict_val += matrix[user_top[0]][key]*user_top[1]
                    div_sum += user_top[1]
            if div_sum != 0:
                predict_val /= div_sum
                recommend_list.append([key,predict_val])
            else:
                recommend_list.append([key,0.0])
        else:
            continue
    return sorted(recommend_list,key = lambda x:x[1],reverse = True)[:n] 

In [212]:
random.seed(99)
custom =  user[random.randint(0,len(user))]
recommend = user_based_cf(custom,user,matrix,10)

[[2393, 5.000000000000001],
 [3283, 5.000000000000001],
 [6, 5.0],
 [63, 5.0],
 [133, 5.0],
 [143, 5.0],
 [246, 5.0],
 [263, 5.0],
 [347, 5.0],
 [442, 5.0]]

In [202]:
custom = user[900]
recommend_list = []
index = user.index(custom) 
list_sim = []
for i in xrange(len(user)):
    if i != index:
        sim = similarity_cos(matrix[i],matrix[index])
        list_sim.append([i,sim])
    else:
        continue
top = [x for x in list_sim if x[1] != 0]
top = sorted(top,key = lambda x:x[1],reverse = True)

In [196]:
len(top)

15096

In [203]:
recommend_list = []
for key,value in enumerate(matrix[index]):
    if value == 0.0:
        predict_val = 0
        div_sum = 0
        for user_top in top:
            if matrix[user_top[0]][key] != 0:
                predict_val += matrix[user_top[0]][key]*user_top[1]
                div_sum += user_top[1]
        if div_sum != 0:
            predict_val /= div_sum
            recommend_list.append([key,predict_val])
        else:
            recommend_list.append([key,0.0])
    else:
        continue
recommend_list = sorted(recommend_list,key = lambda x:x[1],reverse = True)


In [204]:
recommend_list

[[123, 5.0],
 [338, 5.0],
 [732, 5.0],
 [772, 5.0],
 [979, 5.0],
 [1106, 5.0],
 [1186, 5.0],
 [2239, 5.0],
 [2291, 5.0],
 [2698, 5.0],
 [2731, 5.0],
 [3357, 5.0],
 [3369, 5.0],
 [3637, 5.0],
 [3755, 5.0],
 [4032, 5.0],
 [4321, 5.0],
 [4452, 5.0],
 [5014, 5.0],
 [552, 4.395863110478687],
 [16, 4.0],
 [76, 4.0],
 [168, 4.0],
 [361, 4.0],
 [433, 4.0],
 [563, 4.0],
 [711, 4.0],
 [754, 4.0],
 [834, 4.0],
 [943, 4.0],
 [964, 4.0],
 [976, 4.0],
 [1115, 4.0],
 [1144, 4.0],
 [1288, 4.0],
 [1338, 4.0],
 [1470, 4.0],
 [1524, 4.0],
 [1537, 4.0],
 [1603, 4.0],
 [1607, 4.0],
 [1631, 4.0],
 [1778, 4.0],
 [1805, 4.0],
 [1844, 4.0],
 [1894, 4.0],
 [2128, 4.0],
 [2227, 4.0],
 [2235, 4.0],
 [2326, 4.0],
 [2352, 4.0],
 [2446, 4.0],
 [2469, 4.0],
 [2537, 4.0],
 [2608, 4.0],
 [2896, 4.0],
 [3603, 4.0],
 [3624, 4.0],
 [3824, 4.0],
 [4144, 4.0],
 [4602, 4.0],
 [4645, 4.0],
 [4724, 4.0],
 [4981, 4.0],
 [5017, 4.0],
 [3762, 3.8374931441949336],
 [4960, 3.8124106685639405],
 [3283, 3.6331257285365623],
 [282, 3.

In [195]:
m = 0
for i in matrix[1000]:
    if i!= 0:
        m+=1
len(matrix[1000])

5036